In [1]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import torchvision

import numpy as np 
from sklearn.metrics import accuracy_score, roc_auc_score

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from effnetv2_model import EffnetV2Model
from torchsummary import summary

## 0 and 1 dataset

In [3]:
class SimpleDataset(Dataset):
    def __init__(self):
        pass
    
    def __getitem__(self, idx):
        label = np.random.randint(2)
        
        if label == 0:
            img = np.zeros((3,256,256))
        else:
            img = np.ones((3,256,256))
            
        return img, label

    def __len__(self):
        return 1000

In [4]:
simple_dataset = SimpleDataset()

In [5]:
simple_dataloader = DataLoader(simple_dataset, batch_size=4)

In [6]:
model = EffnetV2Model(include_top=True, n_classes=1, model_name='efficientnetv2-s')

In [7]:
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.model = EffnetV2Model(include_top=False)
#         self.classifier = nn.Linear(1280,1)
#     def forward(self, x):
#         x = self.model(x)
#         return self.classifier(x)

In [8]:
# #model = SimpleModel()
# t = torch.rand((1,3,320,320))
# o = model(t)

In [9]:
# o

In [10]:
loss = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [11]:
def train_one_epoch(model, optim, trainloader, loss):
    train_loss = 0 
    all_preds = []
    all_targets = []
    model = model.train()
    for i, (x, y_true) in enumerate(trainloader):
        all_targets.extend(y_true.detach().numpy())
        x = x.float()
        y_true = y_true.unsqueeze(1).float()
        y_pred = model(x)
        l = loss(y_pred, y_true)
        optim.zero_grad()
        l.backward()
        optim.step()
        train_loss += l.item() / len(trainloader)
        if i % 20 == 0:
            print(f'step {i}')
        all_preds.extend(torch.sigmoid(y_pred).detach().numpy()[:,0])
    all_preds_b = (np.array(all_preds)>0.5).astype(int)
    a = accuracy_score(all_targets, all_preds_b)
    r = roc_auc_score(all_targets, all_preds)
    print(train_loss, a, r)
    return model, optim

In [12]:
# for epoch in range(10):
#     model, optim = train_one_epoch(model, optim, simple_dataloader, loss)

## MNIST dataset

In [13]:
mnsit_data = torchvision.datasets.MNIST('../datasets/mnist/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

C:\Work_repository\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [14]:
mnsit_data_test = torchvision.datasets.MNIST('../datasets/mnist/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [15]:
data_loader = torch.utils.data.DataLoader(mnsit_data,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=4)

In [16]:
test_mnist_loader = torch.utils.data.DataLoader(mnsit_data_test,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=4)

In [17]:
len(data_loader), len(test_mnist_loader)

(1875, 313)

In [18]:
model = EffnetV2Model(include_top=True, n_classes=10, n_channels=1)

In [19]:
loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 5e-4)

In [20]:
# trainloader = data_loader
# train_loss = 0 
# all_preds = []
# all_targets = []
# model = model.train()
# for i, (x, y_true) in enumerate(trainloader):
#     all_targets.extend(y_true.detach().numpy())
#     x = x.float()
#     y_true = y_true.long()
#     y_pred = model(x)
#     l = loss(y_pred, y_true)
#     optim.zero_grad()
#     l.backward()
#     optim.step()
#     train_loss += l.item() / len(trainloader)
#     if i % 2000 == 0:
#         print(f'step {i}')
#     preds = torch.nn.functional.softmax(y_pred, dim=0).detach().numpy()
#     preds = np.argmax(preds, axis=1) 
#     all_preds.extend(preds)
#     if i == 10:
#         break
    

# #all_preds_b = (np.array(all_preds)>0.5).astype(int)
# a = accuracy_score(all_targets, all_preds)
# #r = roc_auc_score(all_targets, all_preds)
# print(train_loss, a)


In [21]:
def train_one_epoch(model, optim, trainloader, loss):
    train_loss = 0 
    all_preds = []
    all_targets = []
    model = model.train()
    for i, (x, y_true) in enumerate(trainloader):
        all_targets.extend(y_true.detach().numpy())
        x = x.float()
        y_true = y_true.long()
        y_pred = model(x)
        l = loss(y_pred, y_true)
        optim.zero_grad()
        l.backward()
        optim.step()
        train_loss += l.item() / len(trainloader)
        if i % 1000 == 0:
            print(f'step {i}')
        preds = torch.nn.functional.softmax(y_pred, dim=0).detach().numpy()
        preds = np.argmax(preds, axis=1) 
        all_preds.extend(preds)
        if i == 800:
            break


    #all_preds_b = (np.array(all_preds)>0.5).astype(int)
    a = accuracy_score(all_targets, all_preds)
    #r = roc_auc_score(all_targets, all_preds)
    print('train ', train_loss, a)
    return model, optim

In [22]:
def val_one_epoch(model, optim, valloader, loss):
    val_loss = 0 
    all_preds = []
    all_targets = []
    with torch.no_grad():
        model = model.eval()
        for i, (x, y_true) in enumerate(valloader):
            all_targets.extend(y_true.detach().numpy())
            x = x.float()
            y_true = y_true.long()
            y_pred = model(x)
            l = loss(y_pred, y_true)
            val_loss += l.item() / len(valloader)
            if i % 500 == 0:
                print(f'step {i}')
            preds = torch.nn.functional.softmax(y_pred, dim=0).detach().numpy()
            preds = np.argmax(preds, axis=1) 
            all_preds.extend(preds)
            if i == 500:
                break


    #all_preds_b = (np.array(all_preds)>0.5).astype(int)
    a = accuracy_score(all_targets, all_preds)
    #r = roc_auc_score(all_targets, all_preds)
    print('validation ', val_loss, a)
    

In [23]:
for epoch in range(5):
    print('Epoch ', epoch)
    model, optim = train_one_epoch(model, optim, data_loader, loss)
    val_one_epoch(model, optim, test_mnist_loader, loss)
    print('===========================')

Epoch  0
step 0
step 1000
train  0.27000824939211243 0.8105956543456544
step 0
validation  0.23214488206937095 0.8812
Epoch  1
step 0
step 1000
train  0.10601134681527803 0.9054383116883117
step 0
validation  0.08545216543974872 0.8896
Epoch  2
step 0
step 1000
train  0.06851921037460369 0.9089660339660339
step 0
validation  0.07018668424135774 0.8776
Epoch  3
step 0
step 1000
train  0.08040320037839321 0.8929507992007992
step 0
validation  0.061703788973510046 0.9097
Epoch  4
step 0
step 1000
train  0.05027822950904566 0.9150536963036963
step 0
validation  0.05393782129270225 0.9118
